In [1]:
# General imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#plt.style.use('seaborn-v0_8')
import torch
import os

# MSE function 
def MSE(pred, true):
    return np.mean((pred-true)**2)

random_seed = 100
torch.manual_seed = 100
np.random.seed = 100


In [2]:
from InformerAPI import InformerTS
from CrossformerAPI import CrossformerTS
from AutoformerAPI import AutoformerTS
from FedformerAPI import FedformerTS
from PyraformerAPI import PyraformerTS

### Looping models in one go

In [3]:
datasets = {
    'SYNTHh1': './SYNTHDataset/',
    'DEWINDh_small': './WINDataset/' ,
    'SYNTH_additive' : './SYNTHDataset/' , 
    'SYNTH_additive_reversal' : './SYNTHDataset/' ,
    'SYNTH_multiplicative' : './SYNTHDataset/' ,
    'SYNTH_multiplicative_reversal' : './SYNTHDataset/'
}
pred_lens = [24, 48, 168, 336, 720]

models = [AutoformerTS, FedformerTS]

result_dict = {}

In [4]:
# Simple loop
for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)

    for dataset_name, dataset_path in datasets.items():
        for pred_len in pred_lens:
            print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

            # High amount of epochs to accomodate all models
            # Early stopping should kick in anyways
            model.fit(
                data=dataset_name,
                data_root_path=dataset_path,
                batch_size=32,
                epochs=50, # very high
                pred_len=pred_len
            )

            predictions = model.predict()

            print(f'\nMoving to next...\n')


Model in use: FedformerTS
Training FedformerTS on SYNTHh1 with pred_len 24
Beginning to fit the model with the following arguments:
{'model': 'Fedformer', 'version': 'Fourier', 'mode_select': 'random', 'modes': 64, 'L': 3, 'base': 'legendre', 'cross_activation': 'tanh', 'data': 'SYNTHh1', 'root_path': './SYNTHDataset/', 'data_path': 'SYNTHh1.csv', 'target': 'TARGET', 'checkpoints': './checkpoints/', 'features': 'S', 'freq': 'h', 'detail_freq': 'h', 'seq_len': 96, 'label_len': 48, 'pred_len': 24, 'enc_in': 1, 'dec_in': 1, 'c_out': 1, 'd_model': 512, 'n_heads': 8, 'e_layers': 2, 'd_layers': 1, 'd_ff': 2048, 'moving_avg': 25, 'factor': 1, 'distil': True, 'dropout': 0.05, 'embed': 'timeF', 'activation': 'gelu', 'output_attention': False, 'num_workers': 0, 'itr': 1, 'train_epochs': 50, 'batch_size': 32, 'patience': 3, 'learning_rate': 0.0001, 'des': 'test', 'loss': 'mse', 'lradj': 'type1', 'use_amp': False, 'gpu': 0, 'devices': '0,1,2,3', 'use_multi_gpu': False, 'use_gpu': True}
Use GPU: cu

fourier enhanced block used!
modes=64, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
fourier enhanced block used!
modes=64, index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
 fourier enhanced cross attention used!
modes_q=36, index_q=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35]
modes_kv=48, index_kv=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47]
enc_modes: 48, dec_modes: 36
>>>>>>>start training : Fedformer_SYNTHh1_ftS_sl96_ll48_pl24_dm512_nh8_elNone_dl1_df2048_atNone_fc1_ebtimeF_dtTrue_mxNone_test_1>>>>>>>>>>>>>>>>>>>>>>>>>>
train 85

In [3]:
datasets = {
    'SYNTHh1': './SYNTHDataset/',
    'DEWINDh_small': './WINDataset/' ,
    'SYNTH_additive' : './SYNTHDataset/' ,
    'SYNTH_additive_reversal' : './SYNTHDataset/' ,
    'SYNTH_multiplicative' : './SYNTHDataset/',
    'SYNTH_multiplicative_reversal' : './SYNTHDataset/' 
}
pred_lens = [24, 48, 168, 336, 720]

models = [CrossformerTS , InformerTS]

for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)

    for dataset_name, dataset_path in datasets.items():
        for pred_len in pred_lens:
            print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

            # High amount of epochs to accomodate all models
            # Early stopping should kick in anyways
            model.fit(
                data=dataset_name,
                data_root_path=dataset_path,
                batch_size=32,
                epochs=50, # very high
                pred_len=pred_len
            )

            predictions = model.predict()


            print(f'\nMoving to next...\n')


Model in use: CrossformerTS
Training CrossformerTS on SYNTH_additive_reversal with pred_len 24
Beginning to fit the model with the following arguments:
{'checkpoints': './crossformer_checkpoints/', 'seq_len': 96, 'seg_len': 6, 'win_size': 2, 'factor': 10, 'data_dim': 1, 'd_model': 256, 'd_ff': 512, 'n_heads': 4, 'e_layers': 3, 'dropout': 0.2, 'baseline': False, 'num_workers': 0, 'lradj': 'type1', 'save_pred': True, 'use_gpu': True, 'use_multi_gpu': False, 'gpu': 0, 'devices': '0,1,2,3', 'inverse': False, 'learning_rate': 0.0001, 'loss': 'mse', 'patience': 3, 'data': 'SYNTH_additive_reversal', 'root_path': './SYNTHDataset/', 'data_path': 'SYNTH_additive_reversal.csv', 'train_epochs': 50, 'batch_size': 32, 'pred_len': 24, 'data_split': []}
Use GPU: cuda:0
>>>>>>>start training : Crossformer_SYNTH_additive_reversal_il96_pl24_sl6_win2_fa10_dm256_nh4_el3_itr1>>>>>>>>>>>>>>>>>>>>>>>>>>
train 8521
val 2857
test 2857
	iters: 100, epoch: 1 | loss: 0.2611816
	speed: 0.0722s/iter; left time: 952.

In [6]:
datasets = {
    'SYNTHh1': './SYNTHDataset/',
    'DEWINDh_small': './WINDataset/' ,
    'SYNTH_additive' : './SYNTHDataset/' ,
    'SYNTH_additive_reversal' : './SYNTHDataset/' ,
    'SYNTH_multiplicative' : './SYNTHDataset/',
    'SYNTH_multiplicative_reversal' : './SYNTHDataset/' 
}
pred_lens = [24, 48, 168, 336, 720]

models = [PyraformerTS]

#result_dict_pyraformer = {}

for transformermodel in models:
    print(f'Model in use: {transformermodel.__name__}')
    model = transformermodel()
    model.compile(learning_rate=1e-4, loss='mse', early_stopping_patience=3)

    for dataset_name, dataset_path in datasets.items():
        for pred_len in pred_lens:
            print(f"Training {transformermodel.__name__} on {dataset_name} with pred_len {pred_len}")

            # High amount of epochs to accomodate all models
            # Early stopping should kick in anyways
            model.fit(
                data=dataset_name,
                data_root_path=dataset_path,
                batch_size=32,
                epochs=8, 
                pred_len=pred_len
            )

            #pred_name = 'prediction_' + f'{transformermodel.__name__}_{dataset_name}_{pred_len}'
            #true_name = 'true_' + f'{transformermodel.__name__}_{dataset_name}_{pred_len}'
            

            predictions , trues = model.predict()

            #result_dict_pyraformer[pred_name] = predictions
            #result_dict_pyraformer[true_name] = trues
            

            print(f'\nMoving to next...\n')




Model in use: PyraformerTS
Training PyraformerTS on SYNTH_additive_reversal with pred_len 24
[Info] Number of parameters: 8964864
Data Length: 17420
Train Length: 11520
Test Length:3048
train 11329
Data Length: 17420
Train Length: 11520
Test Length:3048
test 2857
[ Epoch 1 ]


  - (Training) MSE:  0.12463elapse: 0.227 min


test shape:(2848, 24, 1)
Epoch 0, mse:0.030385686084628105, mae:0.13630013167858124, rmse:0.17431490123271942, mape:3.138979911804199, mspe:22122.99609375
[ Epoch 2 ]


  - (Training) MSE:  0.05554elapse: 0.229 min


test shape:(2848, 24, 1)
Epoch 1, mse:0.016809288412332535, mae:0.10002177208662033, rmse:0.12965063750743866, mape:1.862982153892517, mspe:5758.51513671875
[ Epoch 3 ]


...

  - (Training) MSE:  0.16955elapse: 0.217 min


test shape:(2144, 720, 1)
Epoch 6, mse:0.5727700591087341, mae:0.5890669822692871, rmse:0.7568157315254211, mape:2.124436616897583, mspe:177.35447692871094
[ Epoch 8 ]


  - (Training) MSE:  0.16892elapse: 0.215 min


test shape:(2144, 720, 1)
Epoch 7, mse:0.5688241720199585, mae:0.5847705006599426, rmse:0.7542043328285217, mape:2.081162691116333, mspe:164.38644409179688
Iteration best metrics: [0.5643194913864136, 0.5801877975463867, 0.7512120008468628, 2.0586233139038086, 163.13470458984375]
Data Length: 17420
Train Length: 11520
Test Length:3048
train 10633
Data Length: 17420
Train Length: 11520
Test Length:3048
test 2161
168 720


test shape: (2161, 720, 1) (2161, 720, 1)
test shape: (2161, 720, 1) (2161, 720, 1)

Moving to next...

